In [1]:
# ============================================================
# Notebook setup
# ============================================================

%load_ext autoreload
%autoreload 2

# Control figure size
interactive_figures = True
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 5)

data_folder = '/app/data'

# Decision-Focused Learning

A Few Representative Approaches

## Predict, then Optimize

**Industrial optimization problems typically rely on _estimated parameters_**

E.g. travel times, demands, item weights/costs...

<center><img src="assets/traffic.jpg" width="70%"/></center>

**However, sometimes we have access to _a bit more information_**

## Predict, then Optimize

**Take _traffic-dependent travel times_ as an example**

If we know the _time of the day_ we can probably estimate them better!

<center><img src="assets/traffic.jpg" width="70%"/></center>

**Let's see how these problems are typically addressed**

## Predict, then Optimize

**First, we train an estimator for the problem parameters:**

$$
\arg \min \left\{L({\bf y}, {\bf \hat{y}}) \mid {\bf y} = f({\bf \hat{x}}; \theta) \right\}
$$
* $L$ is the loss function
* $f$ is the ML model with parameter 
* ${\bf \hat{x}}$, ${\bf \hat{y}}$ are the training set input/output

**In our example:**

* $x$ would be the time of the day
* $y$ would be a vector of travel times
* $L$ may be a classical MSE loss
* $f$ may be a linear regressor or neural network

## Predict, then Optimize

**Then, we solve the optimization problem with the estimated parameters**

$$
\arg \min_z \left\{ c(z, y) \mid z \in F(y) \right\}
$$
* $z$ is the vector of variables of the optimization problem
* $c$ is the cost function
* $F$ is the feasible space
* In general, both $c$ and $F$ may depend on the estimated parameters

**In our example**

* $z$ may represent routing decisions
* $c$ may be the total travel time
* $F$ may encode a deadline constraint

## Predict, then Optimize

**This approach is sometimes referred to as "Predict, then Optimize"**

It is simple and it makes intuitively sense

* If we manage to estimate well our parameters
* ...The optimization model will be more accurate
* ...And we will obtain better results

**However, things are not really that simple!**

* Our estimator will never be perfect, hence, we will _make mistakes_
* If we are unlucky, these mistakes may lead to _completely wrong decisions_!

**Say we want to move from location A to B**

* There are two possible routes between them
* ...And we need to pick one, based on (e.g.) time of the day

## Predict, then Optimize

**Let us assume this is available data for training**

<center><img src="assets/spo_example_1.png" width="45%"/></center>

* The dashed line shows the input value that causes the optimal choice to switch
* Image from ["Smart Predict, then Optimize"](https://pubsonline.informs.org/doi/pdf/10.1287/mnsc.2020.3922?casa_token=gJcIt01QHyAAAAAA:8WiWc-GVQzS8Jqvp1HO5NkLk-0EoDDcn12yQgtjv3M8--FMr5230ZpoDoOBWH0x4nhHX-YdGjg)

## Predict, then Optimize

**If we train an optimal Linear Regression, we get these estimates**

<center><img src="assets/spo_example_2.png" width="45%"/></center>

* The estimator is quite accurate
* ...But we get the switching point _wrong_!

## Predict, then Optimize

**By contrast, consider this second estimator**

<center><img src="assets/spo_example_3.png" width="45%"/></center>

* The accuracy is awful
* ...But we get the switching point _right_!

## Decision Focused Learning

**The estimator and the optimizer _value mistakes differently_**

* In the estimator we typically care for maximum likelihood
* In the optimization problem, we care about _cost_

> **What if we trained the estimator for _cost minimization_?**

**This is the central idea in _decision-focused learning_**

In principle, the training problem we need to solve is:
$$
\arg \min_{\theta} \left\{ \sum_{i=1}^m c(z_i, y_i) \mid \forall i=1..m : y_i = f(x_i; \theta), z_i = \arg\min_{z \in F(y_i)} c(z, y_i) \right\}
$$

* ...Which is unfortunately hard to tackle, since the $\arg \min$ is non-differentiable

## Sub-gradient to the Rescue

**We could could _fix the estimator output_ $y_i$ in the optimization problem**

* Formally, let $y_i^*$ be the output for example $i$ with the current parameters
* The we consider the following simplified loss:
$$
\tilde{L}(\theta) = \sum_{i=1}^m c(z_i^*, y_i) \\
\text{ with: } y_i = f(x_i; \theta) \text{ and: } z_i^* = \arg\min_{z \in F(y_i^*)} c(z, y_i^*)
$$

Since all $z_i^*$ are now constant, we can obtain a _sub-gradient_:
$$
\nabla_{\theta} \tilde{L}(\theta)
$$
* In an implementation, we compute all (constant) $z_i^*$ in the forward pass...
* ...The we differentiate as usual!

## Sub-gradient to the Rescue

**If you just do this, things often work poorly**

...Because the obtained sub-gradients are often not very good

* This is an area of active research
* With several _open problems_

**Here are a couple of big issues that have been considered:**

* How to take into account the constraints?
  - When the constraints depend on the estimator output $y_i$
  - ...They should be taken into account in the sub-gradient computation!
* How to deal with linear cost functions?
  - They seem easy to handle
  - ...But in fact they are not!

## Dealing with Constraints

**If our constraints are _differentiable_, we can _use a Lagrangian_**

The main idea is to modify the loss to include the constraints as penalty terms:
$$
\mathcal{L}(\theta) = \sum_{i=1}^m \left( c(z_i^*, y_i) + \lambda_i^T g(z_i^*, y_i) \right)
$$
* Where $\lambda_i$ is multiplier vector for example $i$
* ...And all other symbols are the same as before

The resulting sub-gradient include a _constraint-dependent contribution_

**This leaves us with the problem of choosing the multipliers**

* If the optimization problem is _convex_, then [this paper](https://arxiv.org/abs/1703.04529) gives a possible solution

## Dealing with Constraints

**For a broad class of (differentiable) optimization problems**

* Given a local optimum $z_i^*$
* There must exists a multiplier vector such that:

$$\begin{align}
\nabla_{z_i} c(z_i^*, y_i^*) + \sum_{j=1..n} \nabla_{z_i} \lambda_j g_j(z_i^*, y_i^*) = 0 \\
g_j(z_i^*, y_i^*) \leq 0 \quad \forall j = 1..n \\
\lambda_j \geq 0 \quad \forall j = 1..n \\
\lambda_j g_j(z_i^*, y_i^*) = 0 \quad \forall j = 1..n
\end{align}$$

* Where $n$ is the number of problem constraints
* ...And we used the same notation as before, for consistency

**These are known a [Karush-Kuhn Tucker (KKT) optimality conditions](https://en.wikipedia.org/wiki/Karush%E2%80%93Kuhn%E2%80%93Tucker_conditions)**

## Dealing with Constraints

**The KKT conditions may look scary**

* ...But if we know $z_i^*$ already (e.g. because we have already solved the problem)
* Then what we get is just _a system of linear equations in $\lambda$_

$$\begin{align}
\nabla_{z_i} c(z_i^*, y_i^*) + \sum_{j=1..n} \nabla_{z_i} \lambda_j g_j(z_i^*, y_i^*) = 0 \\
g_j(z_i^*, y_i^*) \leq 0 \quad \forall j = 1..n \\
\lambda_j \geq 0 \quad \forall j = 1..n \\
\lambda_j g_j(z_i^*, y_i^*) = 0 \quad \forall j = 1..n
\end{align}$$

* If we solve the system of equations
* ...We can determine the optimal multipliers!

## Dealing with Constraints

**Overall, we perform one training step as follows**

* We compute ${\bf y^*} = f({\bf x}; \theta)$
* We compute $z_i^* = \arg\min_{z \in F(y_i^*)} c(z, y_i^*), \ \forall i=1..m$
* We use the KKT conditions to determine $\lambda_i^T$
* Then we differentiate the Lagrangian loss:
$$
\mathcal{L}(\theta) = \sum_{i=1}^m \left( c(z_i^*, y_i) + \lambda_i^T g(z_i^*, y_i) \right)
$$

**The approach can work well, however**

* It is restricted to convex, differentiable, optimization problems
* It tends to suffer from stability issues
* ...Typically this is mitigated via L2 regularization

## Dealing with Linear Costs

**Linear costs have been mostly studies in a specific context**

...I.e. problems where the constraints _do not depend_ on the estimated parameters

* In these cases, the basis for our sub-gradient is:
$$
\tilde{L}(\theta) = \sum_{i=1}^m y_i^T z_i^* \\
\text{ with: } y_i = f(x_i; \theta) \text{ and: } z_i^* = \arg\min_{z \in F} c(z, y_i^*)
$$
* Same as before, but with linear costs and a fixed feasible set $F$

**This case if of interest for two reasons**

* First, it occurs frequently in practice
* Second, it works even with non-differentiable constraints (e.g. any MIP model)!

## Dealing with Linear Costs

**The problem with linear costs is that they can be made _trivially null_!**

- If we have $y_i = 0$, the cost $y_i^T z_i^*$ becomes null by construction

**The SotA to address this issue is to rely on _surrogate loss functions_**

* For example the [SPO loss](https://pubsonline.informs.org/doi/pdf/10.1287/mnsc.2020.3922?casa_token=gJcIt01QHyAAAAAA:8WiWc-GVQzS8Jqvp1HO5NkLk-0EoDDcn12yQgtjv3M8--FMr5230ZpoDoOBWH0x4nhHX-YdGjg):
$$
\tilde{L}_{\mathit{SPO}}(\theta) = \sum_{i=1}^m (2y_i - \hat{y}_i)^T (\hat{z}_i^* - z_i^*)
$$
* Or one of the NCE losses from [this paper](https://people.cs.kuleuven.be/~tias.guns/files/ijcai21_nce_solpool.pdf):
$$
\tilde{L}_{\mathit{MAP}}(\theta) = \sum_{i=1}^m (y_i - \hat{y}_i)^T (\hat{z}_i^* - z_i^*)
$$
* Where $\hat{z}_i^*$ is the optimum for the true costs $\hat{y}_i$

## Dealing with Linear Costs

**Both surrogate losses have a few common properties**

Let's consider the NCE-MAP loss as an example:
$$
\tilde{L}_{\mathit{MAP}}(\theta) = \sum_{i=1}^m (y_i - \hat{y}_i)^T (\hat{z}_i^* - z_i^*)
$$

* It makes use of the estimated costs $y_i$
  - Otherwise, there would be nothing to differentiate
* It makes use of the true costs $\hat{y}_i$
  - We waste none of the available information
* It is always non-negative (proof skipped)
* It can be minimized by accurately predicting the costs
* It can be minimized by guiding the optimizer toward the true solution

## The Elephant in the Room

**No matter which approach you pick, the is a big issue to deal with**

I.e. computing a subgradient requires to solve an optimization problem

* This may be even NP-hard
* ...And you need to solve one problem per example (so $m$ per epoch)

Meaning that we have considerable _scalability problems_

**A few solutions have been proposed**

* One approach is using [an LP relaxation](https://arxiv.org/pdf/2010.13943) (for MIPs)
* Another simple approach consists in [caching previous solutions](https://people.cs.kuleuven.be/~tias.guns/files/ijcai21_nce_solpool.pdf)
  - Formally, we replace: $z_i^* = \arg\min_{z \in F} c(z, y_i^*)$
  - With: $z_i^* = \arg\min_{z \in S} c(z, y_i^*)$
  - I.e. rather than solving a problem, we pick the best solution from a cache $S$

## Some Results

**Some results for different methods on three problems**

<center><img src="assets/p_and_o.png" width=100%/></center>

* Decision-focused learning improves considerable over the two-stage approach
* All caching-based methods can be quite fast in practice!